## Team 2
- Osama
- Nader
- Mohamed Adel
- Menna
- Aya

# connect 4 game 
- > to run this game you should install pygame 
- > Please run "conda install -c cogsci pygame" in anaconda Prompt


> Note: GUI copied from (https://www.youtube.com/watch?v=UYgyRArKDEs)

In [36]:
import numpy as np
from scipy.signal import convolve2d
import pygame
import sys
import pandas as pd

In [37]:
ROW_COUNT = 6
COLUMN_COUNT = 7
CONNECTED_TO_WIN = 4
SQUARESIZE = 100

colors={'BLUE': (0,0,255),
       'RED':(255,0,0),
       'BLACK':(0,0,0),
       'YELLOW':(255,255,0)}



horizontal_kernel = np.ones((1,CONNECTED_TO_WIN),dtype=np.uint8)
vertical_kernel = np.transpose(horizontal_kernel)
diag1_kernel = np.eye(CONNECTED_TO_WIN, dtype=np.uint8)
diag2_kernel = np.fliplr(diag1_kernel)
win_kernels = [horizontal_kernel, vertical_kernel, diag1_kernel, diag2_kernel]


In [38]:
class Player:
    def __init__(self,name,id_,color):
        self.name=name
        self.id_=id_
        self.color=color
        self.places=np.full((ROW_COUNT,COLUMN_COUNT),-1)
       
    def set_place(self,place):
        self.places[place[0]][place[1]]=self.id_
        
    def checkWin(self,board,win_kernels):
        for kernel in win_kernels:
            if (convolve2d(board == self.places, kernel, mode="valid") == CONNECTED_TO_WIN).any():
                return True
        return False
        

In [39]:
def play(col,player):
    first_empty=(board==0).cumsum(axis=0).argmax(axis=0)
    row=first_empty[col]
    if board[row][col]!=0:
        return (-2,row)
    else:
        board[row][col]=player.id_
        player.set_place((row,col))
        if np.all(board[0]!=0):
            return (0 , row)
        elif player.checkWin(board,win_kernels):
            return (player.id_ , row)
        else:
            return (-1,row)
        
    
    
    

In [40]:
def collectData(data):
    columns=['player','row','column','status']
    new=pd.DataFrame(data,columns=columns)
    try:
        old=pd.read_csv('connect4_data.csv')
        frames=[old,new]
        pd.concat(frames).to_csv('connect4_data.csv',index=False)
    except:
        new.to_csv('connect4_data.csv',index=False)
    

In [41]:
def draw_empty_board(board,screen):
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            pygame.draw.rect(screen, colors['BLUE'], (c*SQUARESIZE, r*SQUARESIZE+SQUARESIZE, SQUARESIZE, SQUARESIZE))
            pygame.draw.circle(screen, colors['BLACK'], (int(c*SQUARESIZE+SQUARESIZE/2), int(r*SQUARESIZE+SQUARESIZE+SQUARESIZE/2)), RADIUS)
    

def draw_board(board,screen,player,r,c):
    if board[r][c] == player.id_:
        pygame.draw.circle(screen, colors[player.color], (int(c*SQUARESIZE+SQUARESIZE/2), (height+SQUARESIZE)-int((ROW_COUNT-r)*SQUARESIZE+SQUARESIZE/2)), RADIUS)


In [42]:
user=Player('osama',1,'RED')
pc=Player('pc',2,'YELLOW')
board=np.full((ROW_COUNT,COLUMN_COUNT),0)

pygame.init()
width = COLUMN_COUNT * SQUARESIZE
height = (ROW_COUNT+1) * SQUARESIZE

size = (width, height)
DATA=[]

RADIUS = int(SQUARESIZE/2 - 5)
myfont = pygame.font.SysFont("monospace", 38)
screen = pygame.display.set_mode(size)
draw_empty_board(board,screen)
pygame.display.update()
GAMEOVER=False
turn=1
while not GAMEOVER:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            GAMEOVER=True
            
        if turn==1:
            if event.type == pygame.MOUSEMOTION:
                pygame.draw.rect(screen, colors['BLACK'], (0,0, width, SQUARESIZE))
                posx = event.pos[0]
                pygame.draw.circle(screen, colors[user.color], (posx, int(SQUARESIZE/2)), RADIUS)
                
            if event.type == pygame.MOUSEBUTTONDOWN:
                pygame.draw.rect(screen, colors['BLACK'], (0,0, width, SQUARESIZE))
                posx = event.pos[0]
                col = int(np.floor(posx/SQUARESIZE))
                key,row=play(col,user)
                print(board)
                print('*************************************************')
                if key==0:
                    label = myfont.render("Game ended with Tie!!", 1, colors['BLUE'])
                    screen.blit(label, (40,10))
                    GAMEOVER=True
                    DATA.append([user.name,row,col,'Tie'])
                    
                elif key==1:
                    label = myfont.render(' You win, congratulation!!!!', 1, colors[user.color])
                    screen.blit(label, (40,10))
                    GAMEOVER=True
                    DATA.append([user.name,row,col,'user win'])
                    
                elif key==-1:
                    turn=2
                    DATA.append([user.name,row,col,'in game'])
           
                draw_board(board,screen,user,row,col)
        elif turn==2:
            col=np.random.randint(0,7)
            key,row=play(col,pc)
            print(board)
            print('*************************************************')
            if key==0:
                    label = myfont.render("Game ended with Tie!!", 1, colors['BLUE'])
                    screen.blit(label, (40,10))
                    GAMEOVER=True
                    DATA.append([pc.name,row,col,'Tie'])
                    
            elif key==2:
                    label = myfont.render(' pc win, good luck next time',1,  colors[pc.color])
                    screen.blit(label, (40,10))
                    GAMEOVER=True
                    DATA.append([pc.name,row,col,'pc win'])
                    
            elif key==-1:
                turn=1
                DATA.append([pc.name,row,col,'in game'])
                
            draw_board(board,screen,pc,row,col)
            
        pygame.display.update()

        
while GAMEOVER:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            del myfont
            pygame.display.quit()
            pygame.quit()
            GAMEOVER=False
            

collectData(DATA)

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]]
*************************************************
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [1 2 0 0 0 0 0]]
*************************************************
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [1 2 0 0 0 0 1]]
*************************************************
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [1 2 0 2 0 0 1]]
*************************************************
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1]
 [1 2 0 2 0 0 1]]
*************************************************
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 2 0 0 1]
 [1 2 0 2 0 0 1]]
*************************************************
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 2 0 0 